<a href="https://colab.research.google.com/github/divyashie/CS201/blob/master/test_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install pyxlsb
!pip install googletrans

In [32]:
import pandas as pd
import pyxlsb
import nltk
from nltk.corpus import brown
import numpy as np
import gensim
import re
# from gensim.models import Word2Vec
# nltk.download()
from googletrans import Translator 

import tensorflow as tf 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Preprocessing

### Cleaning dataset

In [27]:
def cleanData(): 
    path = '/content/gdrive/Shared drives/Thesis2020/final/dataset_Test Report 231.xlsb'
    complaint_df = pd.read_excel(path, engine = 'pyxlsb') #read binary excel file 
    complaint_df = complaint_df.loc[:, (complaint_df != 0).any(axis=0)] #delete all columns with 0s
    complaint_df = complaint_df.dropna(axis=1,how='all') #delete all columns with only NaNs 
    complaint_df = complaint_df.rename(columns=
                    {'osticketcreated': 'ticket_created_date', 
                    'lastmessage':'last_message', 
                    'body' : 'description'})
    sum_nas = complaint_df.isnull().apply(sum,axis=0) 
    for col in complaint_df: 
        if sum_nas[col] >=10000:  #remove columns with more than 1000s nas 
            del complaint_df[col]
    complaint_df = complaint_df.drop(columns = ['sex','ticket_created_date','last_message'],axis=1) #not needed 
    complaint_df = complaint_df[['category','subject', 'description', 'department']]  #reorder columns 
    for column in complaint_df.columns: 
        complaint_df[column] = complaint_df[column].str.replace('[^\w\s]', '') #remove punctuations 
    complaint_df = complaint_df.dropna(how='all')  #remove rows whose fields are just NAs 
    complaint_df = complaint_df.dropna()
    return complaint_df

In [28]:
complaint_df = cleanData()
print(complaint_df.describe())

               category  ...             department
count             25674  ...                  25674
unique              253  ...                    243
top     Street Lighting  ...  Public Infrastructure
freq               4731  ...                   4240

[4 rows x 4 columns]


Translate non-english text to english

In [63]:
def translate_to_english(column): 
  count_english_text =0
  count_french_text =0 
  count_mix_text =0
  saved_translator=[] 
  for row in column: 
      translated = str(translator.detect(row)) 
      if "lang=en" and "confidence=1.0" in translated: 
          count_english_text +=1 
      elif "lang=fr" and "confidence=1.0" in translated:
          count_french_text +=1 
      elif "lang=en" or "lang=fr":
          count_mix_text+=1 #mix language detected
          saved_translator.append(translated)
  print(f"We have {count_english_text} english and {count_french_text} french")


In [64]:
translate_to_english(complaint_df['description'])

ProtocolError: ignored

In [23]:
#Training and validation sets 
X_train, X_test, y_train, y_test = train_test_split(complaint_df['description'], complaint_df['category'], test_size=0.3,random_state=0)
print(f'training: {X_train.shape[0]}')
print(f'validation: {X_test.shape[0]}')

NameError: ignored

In [ ]:
list_wanted_category = [
              'Water',
              'Electricity', 
              'Road Infrastructure',
              'Cleaning',  
              'Animal Welfare',
              'Environmental', 
              'Tax and Licenses', 
              'Consumer Affairs', 
              'Transport', 
              'NHDC housing' 
              'Law & Order',
              'Energy & Public Utilities', 
              'SRM Social Aids', 
              'Sports Infrastructure',
              'Family Welfare', 
              'Financial Services', 
              'Safety & Health',
              'Fisheries & Marine Resources',
              'Industrial Development',
              'Social Aids',
              'Education'
]

### Text Preprocessing

##### Creating a Bag of words representation

Basically, we have to find a word embedding to convert a text into a numerical representtation. Commonly used is BoW model as it learns a vocabulary list from a given corpus and represents each document based on some counting methods of words. 

In [ ]:
def further_cleaning(text, remove_stopwords=False, stemming=False, split_text=False): 
  words = text.lower().split()
  if remove_stopwords: 
     stops = set(stopwords.words("english"))
     words = [w for w in words if not w in stops]
  if stemming == True: 
     stemmer = SnowvallStemmer('english')
     words = [stemmer.stem(w) for w in words]
  if split_text == True: 
     return (words) 
  return (" ".join(words))  #hard coding tokenizer part

In [ ]:
X_train_clean = [further_cleaning(str(data)) for data in X_train]
print(X_train_clean[0])
X_test_clean = [further_cleaning(str(data)) for data in X_test]
print(X_test_clean[0])

Let's fit and transform the training data into a document term matrix with CountVectorizer

In [ ]:
countVec = CountVectorizer()
X_train_countVec = countVec.fit_transform(X_train_clean)
print(len(countVec.get_feature_names())) #Number of features 
print(countVec.get_feature_names()[::1000])

Train a MultinomialNB classifier 

In [ ]:
model_nb = MultinomialNB()
model_nb.fit(X_train_countVec, y_train)

Evaluate model based on the validation set

In [ ]:
pred = model_nb.predict(countVec.transform(X_test_clean))

In [ ]:
print('\nAccuracy {:.4f}'.format(accuracy_score(y_test,pred)))
print('\nClassification report : \n', metrics.classification_report(y_test, pred))
print('\nConfusion matrix: \n', metrics.confusion_matrix(y_test,pred))